# Исследование данных

In [6]:
import sys
# Добавим папку проекта в список системных директорий, чтобы Python видел путь к папке utils
sys.path.append('..')

In [7]:
import pandas as pd

from utils.constants import F
from utils.display_content import (
    display_base_fields_description,
)

from warnings import simplefilter
simplefilter('ignore')

In [8]:
# Загрузим таблицу
data = pd.read_csv('../data/cumulative_life_expectancy_prepared.csv')

data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,BmiTeenagerOverweight2,Sanitation,DrinkingWater,HealthCareCosts,HealthCareCostsGdp,HealthCareCostsPerCapita,GdpPerCapita,Population,Schooling,LifeExpectancy
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,0.99,27.56,41.14,2.74,1.11,7.66,563.733796,16194869.0,5.027188,49.37
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,1.07,28.99,42.25,5.97,2.05,13.35,533.586202,16747208.0,5.092250,50.06
2,AFR,Africa,AGO,Angola,2002,32.51,0.01288,0.758272,29.4,0.027049,...,1.14,30.42,43.38,4.19,1.31,11.54,882.147847,17327699.0,5.157312,51.06
3,AFR,Africa,AGO,Angola,2003,32.16,0.01265,0.676496,29.3,0.026962,...,1.22,31.86,44.36,4.26,1.47,14.55,992.698979,17943712.0,5.222375,51.74
4,AFR,Africa,AGO,Angola,2004,32.24,0.01229,0.594192,29.0,0.026300,...,1.31,33.29,45.35,5.66,1.69,21.36,1266.210864,18600423.0,5.287437,52.36


In [9]:
# Выведем описание полей таблицы
display_base_fields_description()

### Описание полей таблицы

Посмотрим на основные характеристики данных.

In [10]:
print(f"Количество регионов: {data[F.ParentLocationCode.value].nunique()}")

print(f"Количество стран: {data[F.SpatialDimValueCode.value].nunique()}")

year_min = data[F.Period.value].min()
year_max = data[F.Period.value].max()
print(f"Данные представлены за {year_min} - {year_max} гг.")

print(f"Ожидаемая продолжительность жизни \
{data[F.LifeExpectancy.value].min()} - {data[F.LifeExpectancy.value].max()} лет")

Количество регионов: 6
Количество стран: 181
Данные представлены за 2000 - 2020 гг.
Ожидаемая продолжительность жизни 39.96 - 84.66 лет


## Исследование распределения ожидаемой продолжительности жизни

Исследование вынесено в файл [02_target.ipynb](./02_target.ipynb), здесь продублируем выводы.

- Есть тенденция к росту средней ожидаемой продолжительности жизни за рассматриваемый период.
- Наименьшая продолжительность жизни наблюдается в Африке и наибольшая в Европе.
- Среди стран по наибольшей продолжительности жизни лидирует Япония.
- Наибольшую корреляцию с целевой переменной имеет поле `Sanitation` (население, пользующееся как минимум базовыми услугами санитарии (%)) и составляет примерно 0.8.
- Отрицательную корреляцию с ожидаемой продолжительностью жизни имеют признаки, связанные с недостаточной массой тела.

## Кластеризация стран

Поскольку наличие кластера помогает в исследовании данных, \
этот раздел вынесен в блок EDA (исследование данных), \
а не в последующий Feature inginiring (создание признаков).

Описание кластеризации вынесено в файл [03_clustering.ipynb](./03_clustering.ipynb). \
По итогу исследования страны были разделены на два кластера

- В кластер 0 попали страны с более благоприятными условиями \
и соответственно большей продолжительностью жизни.
- А в кластер 1 попапли страны с менее благоприятными условиями \
и меньшей продолжительностью жизни.

Номер кластера занесен в новое поле `ClusterKMeans` \
в основную таблицу `cumulative_life_expectancy_prepared.csv`.

Новый признак кластера имеет корреляцию с целевым близкую к 0.8.

## Исследование влияния факторов, связанных с уровнем санитарии и финансированием медицины на продолжительность жизни

Исследование вынесено в файл [04_medicine.ipynb](./04_medicine.ipynb), здесь продублируем выводы.

- Из рассмотренных признаков на продолжительность жизни наиболее сильно влияет процент населения, у которого есть возможность пользоваться базовыми услугами санитарии (признак `Sanitation`).\
Чем больше этот процент, тем больше продолжительность жизни.

- В среднем по странам процент населения, имеющего доступ к базовым услугам санитарии, увеличивается с течением лет.

- Есть положительная корреляция между расходами на здравоохранение и продолжительностью жизни.

- Процент расходов (от ВВП) на здравоохранение увеличивается с годами.

- Прослеживается закономерность, что чем выше ВВП страны, тем больше она тратит на здравоохранение.\
И тем больше продолжительность жизни в стране. Но есть исключения.

- Чем больше процент расходов на здравоохранение, тем лучше санитарная обстановка в стране.\
Но опять же, есть исключения.

- В странах с более благоприятными условиями жизни (кластер 0) значения всех рассматриваемых показателей больше, чем в странах из кластера 0 (страны с менее благоприятными условиями).

## Исследование связи факторов иммунизациии населения с продолжительностью жизни и причинами детской смертности

Исследование вынесено в файл [05_immunization.ipynb](./05_immunization.ipynb), здесь продублируем выводы.

- Факторы иммунизации имеют отрицательную корреляцию с детской смертностью от инфекционных заболеваний.\
То есть, уменьшают ее.\
Также иммунизация имеет положительную корреляцию с продолжительностью жизни.

- Факторы иммунизации имеют слабую корреляцию с факторами финансирования здравоохранения.

## Исследование факторов, связанных с показателями смертности от разных причин

Исследование вынесено в файл [06_mortality.ipynb](./06_mortality.ipynb), здесь продублируем выводы.

- В странах **кластера 0** выше показатели детской и взрослой смертности от неинфекционных заболеваний.\
Причинами детской смертности преимущественно являются врожденные аномалии и недоношенность.

- В странах **кластера 1** выше показатели материнской смертности и отравлений, а также детской смертности от инфекционных заболеваний.\
Самыми частыми причинами детской смертности являются острые инфекции нижних дыхательных путей, недоношенность, родовая травма и диарея.\
Детская смертность от ВИЧ/CПИД положительно коррелирует со следующими факторами взрослой смертности:
    - Смертность в возрасте от 15 до 60 лет
    - Убийства, самоубийства и сахарный диабет

Для **обоих кластеров**:
- Среди неинфекционных заболеваний самая частая причина смертности - от сердечно-сосудистых заболеваний.
- Материнская смертность положительно коррелирует с детской смертностью от инфекционных заболеваний.

## Исследование факторов, связанных с отклонениями индекса массы тела и употреблением алгоголя

Исследование вынесено в файл [07_health.ipynb](./07_health.ipynb), здесь продублируем выводы.

- В странах с большей продолжительностью жизни (кластер 0) \
больше людей имеют избыточный вес и больше употребляют алкоголя.

- В странах с меньшей продолжительностью жизни (кластер 1) больше людей имеют недостаточный вес. 

- Во всех случаях с избыточным весом отклонений, находящихся в первой стадии (менее выраженной) \
больше, чем во второй стадии (более выраженной).

- Факторы отлонения индекса массы тела в сторону избыточного веса \
имеют слабую корреляцию со смертностью от диабета для стран кластера 0 \
и среднюю (около 0.5) для стран кластера 1.

- Фактор количества употребляемого алкоголя \
имеет наибольшую корреляцию со смертностью от злокачественных новообразований (0.78) и неинфеционных заболеваний (0.6) для стран кластера 0\
и со смертностью от ВИЧ/СПИД для стран кластера 1 (0.4).

## Исследование временных рядов ожидаемой продолжительности жизни по странам

### Декомпозиция 

Поскольку данные ожидаемой продолжительности жизни в каждой стране за рассматриваемые годы являются временным рядом, посмотрим на декомпозицию этих временных рядов.\
Построение графиков с декомпозицией вынесено в файл [08_decomposition.ipynb](./08_decomposition.ipynb).\
Здесь отметим, что ряды имеют только тренд и преимущественно возрастающий.

### Определение стационарности и порядка интегрирования 

Определение стационарности и порядка интегрирования вынесено в файл [09_stationarity.ipynb](./09_stationarity.ipynb).\
С помощью теста Квятковского-Филлипса-Шмидта-Шина (KPSS-тест) определено, что стационарными являются 15 рядов и их графики не являются поступательно-возрастающими, как у большинства стран.\
Путем последовательного дифференцирования определен порядок интергирования рядов и полученные данные сохранены в таблицу `integration_order.csv`.

### Автокорреляция

Построение графиков автокорреляции рядов вынесено в файл [10_autocorrelation.ipynb](./10_autocorrelation.ipynb).\
Для большинства рядов наблюдается наибольшая корреляция для первого лага, что говорит о том, что ряды имеют только тренд.\
Для некоторых стран, ряды продолжительности жизни которых стационарны, явно выделяющихся лагов не наблюдается.\
Это говорит о том, что их значения не зависят от себя в прошлом.

### Автокорреляция и частичная автокорреляция рядов, приведенных к стационарным

Построение графиков вынесено в файл [11_partial_autocorrelation.ipynb](./11_partial_autocorrelation.ipynb).\
Поскольку ряды короткие (а дифференцирование делает ряд еще короче) и ряды не имеют сезонности,\
результат основывается на малом количестве данных.\
Поэтому закономерность в большинстве случаев не улавливается.\
У многих рядов нет явно выступающего лага в обоих графиках.\
Но если он есть, то это преимущественно 1-й лаг.